## Establishing Connection


In [ ]:
from arango import ArangoClient

ArangoClient = ArangoClient(hosts="http://127.0.0.1:8530")

db = ArangoClient.db("yelp_db", username="super", password="grantaccess")

collections = db.collections()


## Preprocess Json data for easy loading

In [ ]:
import json

def preprocess_json(input_file, output_file):
    with open(input_file, 'r') as f:
        # Read the entire file content
        data = f.read()

        # Split the content by newline character to handle multiple JSON objects
        json_objects = data.strip().split('\n')

    # Process each JSON object separately
    processed_data = []
    for json_str in json_objects:
        try:
            # Load each JSON object separately
            obj = json.loads(json_str)
            processed_data.append(obj)
        except json.JSONDecodeError as e:
            print("Error decoding JSON:", e)

    # Write processed data to the output file
    with open(output_file, 'w') as f:
        json.dump(processed_data, f)

if __name__ == "__main__":
    input_file = "yelp_academic_dataset_user.json"  # Replace with the path to your input JSON file
    output_file = "yelp_academic_dataset_user_updated.json"  # Replace with the path to your output JSON file
    preprocess_json(input_file, output_file)


In [ ]:
import json
collection_name = "user"
collection=db.collection(collection_name)
# Path to the JSON file
json_file = "yelp_academic_dataset_user_updated.json"  

# Read the JSON data from file
with open(json_file, 'r') as f:
    data = json.load(f)

# Bulk insert the data into the collection
collection.import_bulk(data)

## Defining Edge collection based on uploaded Collection

In [ ]:
from arango import ArangoClient

# Connect to ArangoDB
client = ArangoClient(hosts='http://127.0.0.1:8530')
db = client.db('yelp_db', username='root', password='')



In [ ]:
tipscollection=db.collection('tip')

cursor = tipscollection.all()

In [ ]:
# Your function to retrieve records and create edge documents
def process_records(cursor):
    edge_documents = []
    for record in cursor:
        # Assuming each record is a dictionary
        # Generate edge document for business
        edge_documents.append({
            "_from": f"tips/{record['_key']}",  # Assuming '_key' is the document key
            "_to": f"business/{record['business_id']}"
        })
    return edge_documents

# Process the cursor to generate edge documents
edge_documents = process_records(cursor)

In [ ]:
print(len(edge_documents))

In [ ]:
# Function to insert records into an edge collection in batches
def insert_edges(collection_name, edge_documents, batch_size=1000):
    for i in range(0, len(edge_documents), batch_size):
        batch = edge_documents[i:i+batch_size]
        db[collection_name].import_bulk(batch)

# Split the edge documents into batches and insert them into the collection
# insert_edges('tip_business', edge_documents)


In [ ]:
cursor = tipscollection.all()

In [ ]:
def process_records_user(cursor):
    edge_documents = []
    for record in cursor:
        edge_documents.append({
            "_from": f"tips/{record['_key']}",  # Assuming '_key' is the document key
            "_to": f"user/{record['user_id']}"
        })
    return edge_documents


edge_documents_user = process_records_user(cursor)

In [ ]:
print(len(edge_documents_user))

In [ ]:
insert_edges('tip_user', edge_documents_user)

In [ ]:
checkinscollection=db.collection('checkin')
checkin_records = checkinscollection.all()

In [ ]:
print(len(checkin_records))

In [ ]:
def process_records_checkin_business(cursor):
    edge_documents = []
    for record in cursor:
        edge_documents.append({
            "_from": f"checkin/{record['_key']}",  # Assuming '_key' is the document key
            "_to": f"business/{record['business_id']}"
        })
    return edge_documents

In [ ]:
edge_documents_checkin = process_records_checkin_business(checkin_records)

In [ ]:
insert_edges('checkin_business', edge_documents_checkin)

In [ ]:
def process_records_review_business(cursor):
    edge_documents = []
    for record in cursor:
        edge_documents.append({
            "_from": f"review/{record['_key']}",  # Assuming '_key' is the document key
            "_to": f"business/{record['business_id']}"
        })
    return edge_documents
def process_records_review_user(cursor):
    edge_documents = []
    for record in cursor:
        edge_documents.append({
            "_from": f"review/{record['_key']}",  # Assuming '_key' is the document key
            "_to": f"user/{record['user_id']}"
        })
    return edge_documents

In [ ]:
reviewsCollection=db.collection('review')

review_records = reviewsCollection.all()


In [ ]:
print(len(review_records))

In [ ]:
type(review_records)

In [ ]:
edge_documents_review_business = process_records_review_business(review_records)

In [ ]:
insert_edges('review_business', edge_documents_review_business)

In [ ]:
edge_documents_review_user = process_records_review_user(review_records)

In [ ]:
insert_edges('review_user', edge_documents_review_user)

## Analysis on Data


In [2]:
from arango import ArangoClient

ArangoClient = ArangoClient(hosts="http://192.168.1.120:8531")

db = ArangoClient.db("yelp_db", username="super", password="grantaccess")

# Categories of Business and their count

In [3]:
#fetch and plot different categories of businesses

categories=db.aql.execute("FOR b IN business RETURN b.categories")

import matplotlib.pyplot as plt
import pandas as pd
categories_df = pd.DataFrame(categories, columns=['categories'])
print(categories_df.head())


                                          categories
0  Doctors, Traditional Chinese Medicine, Naturop...
1  Shipping Centers, Local Services, Notaries, Ma...
2  Department Stores, Shopping, Fashion, Home & G...
3  Restaurants, Food, Bubble Tea, Coffee & Tea, B...
4                          Brewpubs, Breweries, Food


In [4]:
import pandas as pd
import plotly.graph_objects as go

cat_list = []
for i in categories_df['categories']:
    if i:
        for j in i.split(","):
            cat_list.append(j.strip())
cat_df = pd.DataFrame(cat_list, columns=['categories'])
x = cat_df['categories'].value_counts()

print("There are ", len(x), " different types/categories of Businesses in Yelp!")

# Select the top 20 categories
x = x.sort_values(ascending=False).iloc[:20]

# Define colors for each bar
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
          '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf',
          '#ff0000', '#00ff00', '#0000ff', '#ffff00', '#ff00ff',
          '#00ffff', '#800000', '#008000', '#000080', '#808000']

# Create a Plotly bar chart
fig = go.Figure(data=[
    go.Bar(
        x=x.index,
        y=x.values,
        marker_color=colors
    )
])

fig.update_layout(
    title="What are the top categories?",
    xaxis=dict(title='Category'),
    yaxis=dict(title='# businesses'),
    xaxis_tickangle=-45,
    width=1000,
    height=500
)

fig.show()


There are  1311  different types/categories of Businesses in Yelp!


# Star rating of business

In [5]:
# fetch from buiness collection the star rating of each business
star=db.aql.execute("FOR b IN business RETURN b.stars")
star_df = pd.DataFrame(star, columns=['stars'])
import plotly.graph_objs as go
import pandas as pd

# Assuming star_df is your DataFrame containing the 'stars' column
# Replace 'star_df' with the name of your DataFrame if different

# Step 1: Get the value counts of stars
star_counts = star_df['stars'].value_counts().sort_index()

# Step 2: Create the Plotly bar plot
data = [
    go.Bar(
        x=star_counts.index,
        y=star_counts.values,
        marker=dict(color='rgb(158,202,225)')
    )
]

layout = go.Layout(
    title='Distribution of Star Ratings',
    xaxis=dict(title='Stars'),
    yaxis=dict(title='Count'),
    bargap=0.2,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
fig.show()


In [11]:
import plotly.graph_objs as go
import pandas as pd

aql = """
    FOR r IN review
        LET date = DATE_ISO8601(DATE_TIMESTAMP(r.date))
        LET year = DATE_YEAR(date)
        COLLECT review_year = year
        WITH COUNT INTO review_count
        SORT review_year ASC
        RETURN { year: review_year, review_count: review_count }
"""

# Execute the query
cursor = db.aql.execute(aql)

# Prepare data for visualization
data = [(doc['year'], doc['review_count']) for doc in cursor]
years, counts = zip(*data)

# Convert data to DataFrame
df = pd.DataFrame({'Year': years, 'Review_Count': counts})

# Sort DataFrame by Year
df['Year'] = pd.to_datetime(df['Year'], format='%Y')
df.sort_values(by='Year', inplace=True)

# Create Plotly figure
fig = go.Figure()

# Add bar trace
fig.add_trace(go.Bar(
    x=df['Year'],
    y=df['Review_Count'],
    marker=dict(color=df['Year'].dt.year),
    text=df['Review_Count'],
    textposition='outside',
))

# Update layout
fig.update_layout(
    title='Number of Reviews per Year',
    xaxis_title='Year',
    yaxis_title='Number of Reviews',
    xaxis=dict(tickformat='%Y'),
    yaxis=dict(showgrid=True),
)

# Show plot
fig.show()


In [ ]:
import folium
import pandas as pd

# Define your AQL query to retrieve latitude, longitude, and business name
aql_query = """
    FOR b IN business
    LIMIT 500
        RETURN { 
            name: b.name,
            latitude: b.latitude,
            longitude: b.longitude
        }
"""

# Execute the AQL query to retrieve business data
cursor = db.aql.execute(aql_query)

# Prepare data for visualization
data = [(doc['name'], doc['latitude'], doc['longitude']) for doc in cursor]
business_names, latitudes, longitudes = zip(*data)

# Create a DataFrame
df = pd.DataFrame({'Business': business_names, 'Latitude': latitudes, 'Longitude': longitudes})

# Create a base map
m = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=5)

# Add markers for each business location
for index, row in df.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], popup=row['Business']).add_to(m)

# Display the map
m
